# ConversationBufferWindowMemory

`ConversationBufferWindowMemory` 는 시간이 지남에 따라 대화의 상호작용 목록을 유지합니다.

이때, `ConversationBufferWindowMemory` 는 모든 대화내용을 활용하는 것이 아닌 **최근 K개** 의 상호작용만 사용합니다.

이는 버퍼가 너무 커지지 않도록 가장 최근 상호작용의 슬라이딩 창을 유지하는 데 유용할 수 있습니다.


In [1]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, return_messages=True) # [message] if return_messages else [str] 

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)
memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)
memory.save_context(
    inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
    outputs={
        "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)
memory.save_context(
    inputs={"human": "인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?"},
    outputs={
        "ai": "본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "정보를 모두 입력했습니다. 다음 단계는 무엇인가요?"},
    outputs={
        "ai": "입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요."
    },
)
memory.save_context(
    inputs={"human": "모든 절차를 완료했습니다. 계좌가 개설된 건가요?"},
    outputs={
        "ai": "네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!"
    },
)

대화기록을 확인해 보면 **최근 2개** 의 메시지만 반환하는 것을 확인할 수 있습니다.


In [2]:
# 대화 기록을 확인합니다.
memory.load_memory_variables({})["history"]

[HumanMessage(content='정보를 모두 입력했습니다. 다음 단계는 무엇인가요?'),
 AIMessage(content='입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요.'),
 HumanMessage(content='모든 절차를 완료했습니다. 계좌가 개설된 건가요?'),
 AIMessage(content='네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!')]

## Chain 에 적용

In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [13]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH05-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


In [14]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

# LLM 모델을 생성합니다.
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# ConversationChain을 생성합니다.
conversation = ConversationChain(
    # ConversationBufferMemory를 사용합니다.
    llm=llm,
    memory = ConversationBufferWindowMemory(k=2, return_messages=False) # [message] if return_messages else [str] ,
)

In [15]:
# 대화를 시작합니다.
response = conversation.predict(
    input="안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
)
print(response)

안녕하세요! 은행 계좌를 개설하려면 먼저 해당 은행의 공식 웹사이트에 접속하셔야 합니다. 거기서 온라인 계좌 개설 서비스를 찾아보시면 됩니다. 보통 개인 정보, 신분증 사본, 주소지 확인서 등을 제출해야 합니다. 그리고 온라인 양식을 작성하고 필요한 정보를 입력하면 계좌 개설이 완료됩니다. 만약 도움이 필요하시다면 은행 고객센터에 문의하시는 것도 좋은 방법입니다. 혹시 어떤 은행을 이용하실 건가요?


In [16]:
response = conversation.predict(
    input= "기업은행을 이용하려고 합니다. 다음으로 무엇을 해야 하나요?"
)
print(response)

기업은행을 이용하시려면 먼저 기업은행의 공식 웹사이트에 접속하셔야 합니다. 거기서 온라인 계좌 개설 서비스를 찾아보시면 됩니다. 보통 개인 정보, 신분증 사본, 주소지 확인서 등을 제출해야 합니다. 그리고 온라인 양식을 작성하고 필요한 정보를 입력하면 계좌 개설이 완료됩니다. 만약 도움이 필요하시다면 기업은행 고객센터에 문의하시는 것도 좋은 방법입니다. 계좌 개설 후에는 인터넷 뱅킹 서비스를 통해 편리하게 계좌를 이용하실 수 있습니다. 계좌 개설 과정에서 어려움이 있으시면 언제든지 저에게 물어보세요!


In [17]:
response = conversation.predict(
    input= "인터넷뱅킹의 장점은 무엇인가요?"
)
print(response)

인터넷뱅킹의 장점은 여러 가지가 있습니다. 먼저, 언제 어디서나 24시간 내내 은행 업무를 처리할 수 있어 편리합니다. 또한, 온라인을 통해 계좌 잔액 확인, 이체, 결제, 대출 신청 등 다양한 은행 업무를 간편하게 처리할 수 있습니다. 더불어, 종이로 된 명세서나 영수증을 받지 않아도 되어 환경에도 도움이 됩니다. 또한, 보안 시스템이 강화되어 있어 안전하게 거래를 할 수 있습니다. 이렇게 인터넷뱅킹은 효율적이고 편리한 서비스를 제공합니다.


In [18]:
response = conversation.predict(
    input= "카카오뱅킹으로 옮겨야 겠네요."
)
print(response)

카카오뱅킹으로 옮기시려면 먼저 카카오뱅킹 앱을 다운로드하셔야 합니다. 그 후에는 앱을 실행하여 계좌 이체나 개설을 진행하실 수 있습니다. 카카오뱅킹은 간편하고 빠른 서비스로 유명하며, 다양한 혜택과 편의성을 제공합니다. 이용하시면서 편리함을 느끼실 것입니다. 만약 도움이 필요하시다면 카카오뱅킹 고객센터에 문의하시는 것도 좋은 방법입니다. 계좌 이체나 개설 과정에서 어려움이 있으시면 언제든지 저에게 물어보세요!


# langsmith 확인!!!